In [1]:
import sys

sys.path.append('/home/max/myrepos/vlstm_cuda/')

import torch

In [2]:
from src_triton.fused_attention.triton_tutorial import attention

In [3]:
S = 128 #32 #32 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 64 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(1) #TODO from here: with seed=0 even the pytorch version alone breaks for float16 and bfloat16
# fixed:
# qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
# qs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# # vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# vs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 100.
# # vs = torch.zeros((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# vs[:,:,1,0] = 7.
# qs[:,:,1,0] = 1.

# vs[:,:,1,16] = 8.
# qs[:,:,1,16] = 1.
# random: 
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
# fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)

dHs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)

In [5]:
# inputs float16
dtype_fp16 = torch.float16
qs_half = qs.to(dtype=dtype_fp16)
ks_half = ks.to(dtype=dtype_fp16)
vs_half = vs.to(dtype=dtype_fp16)
igs_half = igs.to(dtype=dtype_fp16)
fgs_half = fgs.to(dtype=dtype_fp16)
dHs_half = dHs.to(dtype=dtype_fp16)

In [6]:
# inputs bfloat16
dtype_bf16 = torch.bfloat16
qs_bf16 = qs.to(dtype=dtype_bf16)
ks_bf16 = ks.to(dtype=dtype_bf16)
vs_bf16 = vs.to(dtype=dtype_bf16)
igs_bf16 = igs.to(dtype=dtype_bf16)
fgs_bf16 = fgs.to(dtype=dtype_bf16)
dHs_bf16 = dHs.to(dtype=dtype_bf16)

In [8]:
attention(qs_half, ks_half, vs_half, True, 0.5)

tensor([[[[-0.5811, -1.0215,  0.8330,  ...,  1.2637,  0.4214,  0.2869],
          [ 0.2651,  0.0389,  0.6138,  ...,  1.9795,  0.9600,  1.1768],
          [ 0.6157,  1.3809,  0.4976,  ...,  1.0830,  1.0732, -0.0125],
          ...,
          [-0.1365,  0.2446, -0.0297,  ..., -0.2944,  0.6147, -0.0832],
          [-0.0317, -0.3557, -0.5635,  ...,  0.0829,  0.2019,  1.0996],
          [-0.2773,  0.7500, -0.0801,  ..., -0.0390, -0.3489, -1.4082]]]],
       device='cuda:0', dtype=torch.float16)

In [9]:
torch.nn.functional.scaled_dot_product_attention(qs_half, ks_half, vs_half, is_causal=True, scale=0.5)

tensor([[[[-0.5811, -1.0215,  0.8330,  ...,  1.2637,  0.4214,  0.2869],
          [ 0.2651,  0.0389,  0.6138,  ...,  1.9795,  0.9600,  1.1768],
          [ 0.6157,  1.3809,  0.4976,  ...,  1.0830,  1.0732, -0.0125],
          ...,
          [-0.1365,  0.2446, -0.0297,  ..., -0.2944,  0.6147, -0.0832],
          [-0.0317, -0.3557, -0.5635,  ...,  0.0829,  0.2019,  1.0996],
          [-0.2773,  0.7500, -0.0801,  ..., -0.0390, -0.3489, -1.4082]]]],
       device='cuda:0', dtype=torch.float16)